In [1]:
import os

In [2]:
import zipfile
import requests
from typing import Tuple
from google.cloud import storage
from datetime import timedelta
from typing import List, Tuple
from prefect import flow, task
from prefect.tasks import task_input_hash, exponential_backoff

In [3]:
#@task(name='download_data', retries=3, retry_delay_seconds=exponential_backoff(backoff_factor=10), cache_key_fn=task_input_hash, cache_expiration=timedelta(hours=2))
def download_data(url: str, output_dir: str) -> str:

    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, os.path.basename(url))

    response = requests.get(url)
    with open(file_path, 'wb') as f:
        f.write(response.content)

    return file_path

In [11]:
os.makedirs('data')

FileExistsError: [Errno 17] File exists: 'data'

In [ ]:
url = 'https://dataverse.harvard.edu/api/access/dataset/:persistentId/?persistentId=doi:10.7910/DVN/HG7NV7',
output_dir = '../data',
bucket_name = 'airline-buckets'